All paths as in properties.py but with changed UNIX style paths for the code to execute across platforms

In [1]:
import os 
V={}
WORK_DIR = '/media/hru/Data/_Active_Projects/PythonRDAHMM/test'
V['cron_path']=os.path.join(WORK_DIR, "RDAHMM","CRON_Download/")  
V['download_path']=os.path.join(WORK_DIR,"RDAHMM","Download/")  
V['script_path']=os.path.join(WORK_DIR,"PythonRDAHMM/") 
V['data_path']=os.path.join(WORK_DIR,"RDAHMM","Data/")
# temp_path is the temporary working directory for ingesting raw data
V['temp_path']=os.path.join(WORK_DIR,"RDAHMM","TEMP/")
V['model_path']=os.path.join(WORK_DIR,"RDAHMM","Model/")
V['eval_path']=os.path.join(WORK_DIR,"daily/single/")
V['train_epoch']="2013-12-31"
V['rdahmm_bin']=os.path.join(WORK_DIR,"RDAHMM", "rdahmm3","bin", "rdahmm")
V['rdahmm_model_parm']="-data <inputFile> -T <dataCount> -D <dimensionCount> -N 5 -output_type gauss -anneal -annealfactor 1.1 -betamin 0.1 -regularize -omega 0 0 1 1.0e-6 -ntries 10 -seed 1234"
V['rdahmm_eval_parm']="-data <proBaseName>.all.input -T <dataCount> -D <dimensionCount> -N 5 -output_type gauss -A <modelBaseName>.A -B <modelBaseName>.B -pi <modelBaseName>.pi -minvalfile <modelBaseName>.minval -maxvalfile <modelBaseName>.maxval -rangefile <modelBaseName>.range -eval"
V['dygraphsJs']=os.path.join(WORK_DIR,"PythonRDAHMM","dygraphsJsCreator.pearl")

def properties(key):
    return V[key]

# Generate the json file for Presentation layer
This is the existing code obtained from PythonRDAHMM/create_summary_jsons.py 

In [2]:
import os, sys, string, re, json
from datetime import date, datetime, timedelta, time

In [3]:
# command line input argument
# dataSet = "rawNeuTimeSeries.MEASURES_Combination"
dataSet = "UNR_SPLICE"

In [4]:
# Some useful global constants
today = datetime.today()
serverName = "gf9.ucs.indiana.edu"
updateTime = str(today.strftime("%Y-%m-%dT%H:%M:%S"))
beginDate = "1994-01-01"
# endDate = str(today.strftime("%Y-%m-%d"))
endDate = '2016-10-19'
centerLng = "-119.7713889"
centerLat = "36.7477778"
stateChangeNumTxtFile = "stateChangeNums.txt"
stateChangeNumJsInput = "stateChangeNums.txt.jsi"
allStationInputName = "all_stations.all.input"
filters = "Fill_Missing"

In [5]:
# Used to separate parts of the station name
SEPARATOR_CHARACTER="_"
NO_DATA_TIME_STAMP="22:22:22"
FINAL_PATH=properties('eval_path')

In [6]:
def setStationId(stationList, stationData):
    #Get the station name.
    stationName=stationList.split(SEPARATOR_CHARACTER)[2];

    stationData['id'] = stationName
    stationData['pro_dir'] = "daily_project_" + stationName + "_" + endDate
    stationData['AFile'] = "daily_project_" + stationName + ".A"
    stationData['BFile'] = "daily_project_" + stationName + ".B"
    stationData['InputFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input"
    stationData['RawInputFile'] = "daily_project_" + stationName + "_" + endDate + ".all.raw"
    stationData['SwfInputFile'] = "daily_project_" + stationName + "_" + endDate + ".plotswf.input"
    stationData['DygraphsInputFile'] = "daily_project_" + stationName + "_" + endDate + ".dygraphs.js"
    stationData['LFile'] = "daily_project_" + stationName + ".L"
    stationData['XPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.X.png"
    stationData['YPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.Y.png"
    stationData['ZPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.Z.png"
    stationData['XTinyPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.X_tiny.png"
    stationData['YTinyPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.Y_tiny.png"
    stationData['ZTinyPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.Z_tiny.png"
    stationData['PiFile'] = "daily_project_" + stationName + ".pi"
    stationData['QFile'] = "daily_project_" + stationName + "_" + endDate + ".all.Q"
    stationData['MaxValFile'] = "daily_project_" + stationName + ".maxval"
    stationData['MinValFile'] = "daily_project_" + stationName + ".minval"
    stationData['RangeFile'] = "daily_project_" + stationName + ".range"
    stationData['ModelFiles'] = "daily_project_" + stationName + ".zip"
    stationData['RefFile'] = "daily_project_" + stationName + ".input.ref"
    return

In [7]:
def setStationStartDate(stationDir, stationData):
    startFileName = stationDir + "daily_project_" + stationData['id'] + ".input.starttime"
    if (os.path.isfile(startFileName)):
        with open(startFileName,"r") as startFile:
            startDate = startFile.readline().rstrip()
        startFile.close()
    else:
		startDate = "1994-01-01"
    stationData['start_date'] = startDate
    return

In [8]:
def setStationRefLatLonHgt(stationDir, stationData):
    refFileName = stationDir + stationData['RefFile']
    refLat=""
    refLon=""
    refHgt=""
    if (os.path.isfile(refFileName)):
        with open(refFileName,"r") as refFile:
            refParts=refFile.readline().split(" ")
            refLat=refParts[0]
            refLon=refParts[1]
            refHgt=refParts[2].rstrip() # Have to chomp off the final \n
        refFile.close()
    else:
        refLat="1.0"
        refLon="2.0"
        refHgt="-1.0"

    stationData['lat'] = refLat      
    stationData['long'] = refLon      
    stationData['height'] = refHgt
    return

In [9]:
def setStatusChanges(stationDir, stationData):
    # Open the .all.Q and the .all.raw files.  We get the state from the first and
    # the data from the second. 
    # TODO: for now, we assume these files always exist
    qFileName = stationDir + stationData['QFile']
    rawFileName = stationDir + stationData['RawInputFile']
   



    # Bail out if the required files don't exist   
    if((not os.path.isfile(qFileName)) or (not os.path.isfile(rawFileName))): 
        return 
    qFile = open(qFileName,"r")
    rawFile = open(rawFileName,"r")

    stateChanges = []
    changeCount = 0
    # Now step through the Q file looking for state changes
    # If we find a state change, get the date from the raw file
    # We will save these to the string stateChangeArray since we
    # need to record in latest-first order
    qline1 = qFile.readline()
    rline1 = rawFile.readline()        
    while True:
        eventData = {}
        qline2 = qFile.readline()
        rline2 = rawFile.readline()
        if not qline2: break
        
        # See if qline1 and qline2 are the same.  If so, extract the dates from rline1 and rline2
        # The line splits below are specific to the raw file line format.
        if (qline1.rstrip() != qline2.rstrip()):
            eventdate = rline2.split(" ")[1] 
            eventdate = eventdate.split("T")[0]
            oldstate = qline1.rstrip()
            newstate = qline2.rstrip()
            eventData['date'] = eventdate
            eventData['from'] = oldstate
            eventData['to'] = newstate 
            stateChanges.append(eventData)
            changeCount += 1

        # Make the previous "next" lines the "first" lines for the next comparison
        qline1=qline2
        rline1=rline2

    stationData['status_changes'] = stateChanges
    stationData['change_count'] = changeCount

    # Clean up
    qFile.close
    rawFile.close
    return

In [10]:
def setTimesNoData(stationDir, stationData):
    rawFileName = stationDir + stationData['RawInputFile']

    # Required file doesn't exist so bail out
    if(not os.path.isfile(rawFileName)): return
    rawFile = open(rawFileName, "r")
    
    noDataRanges = []
    noDataCount = 0
    noDataEvent = {}

    # We need to set a no-data range from beginDate (for the epoch, 1994-01-01) to the day before
    # our first data point for this station.  If the station has data before 1994-01-01, then 
    # ignore.
    firstDataDateParts=rawFile.readline().split(" ")[1].split("T")[0].split("-");

    beginEpoch=date(1994,1,1)

    #Convert this into a data object
    dayMinusOne=date(int(firstDataDateParts[0]),int(firstDataDateParts[1]),int(firstDataDateParts[2]))
    dayMinusOne-=timedelta(days=1)
    if(dayMinusOne > beginEpoch): 
        dayMinusOneString=dayMinusOne.isoformat()
        noDataEvent['to'] = dayMinusOneString
        noDataEvent['from'] = beginDate
        noDataCount += 1

    #Reset the "raw" file to the beginning
    rawFile.seek(0)

    # Step through the file to find the starting and ending dates with no data.
    # By convention, this occurs when the line has a timestamp T22:22:22.  Also, by
    # convention, we will record the latest to earliest dates with no data.

    while True:
        noDataEvent = {}
        nodata=False
        rline1=rawFile.readline()
        if not rline1: break

        # Get the date and timestamp, following format conventions
        fulleventdate1=rline1.split(" ")[1]
        eventdate1=fulleventdate1.split("T")[0]
        timestamp1=fulleventdate1.split("T")[1]

        # See if we have detected a no-data line
        if(timestamp1==NO_DATA_TIME_STAMP):
            nodata=True
            #Keep eventdate1 in case this is an isolated no-data line.
            eventdate_keep=eventdate1

            # We have a no-data line, so step ahead until the 
            # no-data line ends.
            while(nodata):
                rline2=rawFile.readline()
                if not rline2: break
                fulleventdate2=rline2.split(" ")[1]
                eventdate2=fulleventdate2.split("T")[0]
                timestamp2=fulleventdate2.split("T")[1]
                if(timestamp2!=NO_DATA_TIME_STAMP):
                    # Data exists for the second time stamp, so break out
                    # The last no-data line was the previous line
                    nodata=False
                    break
                else:
                    # No data for this line either, so keep this timestamp
                    # and start the while(nodata) loop again
                    eventdate_keep=eventdate2

            # We now know the range of no-data values, so insert this range, latest first
	    noDataEvent['to'] = eventdate_keep
	    noDataEvent['from'] = eventdate1
	    noDataRanges.append(noDataEvent)
	    noDataCount += 1
            
    # Finally, prepend the data-not-yet-available date range, from the last day of data
    # until today's date.
    today=date.today()
    formattedToday=today.isoformat() 
    
    #Reread the last event
    rawFile.seek(0)
    lastRawLine=rawFile.readlines()[-1]
    lastRawDate=lastRawLine.split(" ")[1].split("T")[0]
    lastDataDateParts=lastRawDate.split("-")  # This is the last date
    #Create a new date object out of the string we get from the file.
    lastDataDatePlus1=date(int(lastDataDateParts[0]),int(lastDataDateParts[1]),int(lastDataDateParts[2]))
    #Now increment this date one day.
    lastDataDatePlus1+=timedelta(days=1)    
    #Now convert to a string
    lastDataDataP1String=lastDataDatePlus1.isoformat()

    noDataEvent = {}
    noDataEvent['to'] = formattedToday
    noDataEvent['from'] = lastDataDataP1String
    noDataRanges.append(noDataEvent)
    noDataCount += 1
    
    stationData['time_nodata'] = noDataRanges
    stationData['nodata_count'] = noDataCount
    rawFile.close
    return

In [11]:
projectDir = FINAL_PATH +dataSet
if(os.path.isdir(projectDir)):
# Open the JSON file that will contain the results
    outputPath = FINAL_PATH+dataSet + "_FILL.json"
    summaryData = {}
        
    summaryData['update_time'] = updateTime
    summaryData['data_source'] = dataSet
    summaryData['begin_date'] = beginDate
    summaryData['end_date'] = endDate
    summaryData['center_longitude'] = centerLng
    summaryData['center_latitude'] = centerLat
    summaryData['server_url'] = "http://" + serverName + "/daily_rdahmmexec/daily/" + dataSet
    summaryData['stateChangeNumTxtFile'] = stateChangeNumTxtFile
    summaryData['stateChangeNumJsInput'] = stateChangeNumJsInput
    summaryData['allStationInputName'] = allStationInputName
    summaryData['Filters'] = filters
    summaryData['video_url'] = ""

    stations = []
    stationCount = 0

In [12]:
outputPath

import os, errno

def silentremove(filename):
    try:
        os.remove(filename)
    except OSError as e: # this would be "except OSError, e:" before Python 2.6
        if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
            raise # re-raise exception if a different error occured
            
silentremove(outputPath)

In [13]:
for stationList in os.listdir(projectDir):
    stationPath = projectDir + "/" + stationList + "/"
    if (os.path.isdir(stationPath)):
        stationData = {}

        setStationId(stationList, stationData)
        setStationStartDate(stationPath, stationData)
        setStationRefLatLonHgt(stationPath, stationData)
        setStatusChanges(stationPath, stationData)
        setTimesNoData(stationPath, stationData)

        stations.append(stationData)
        stationCount += 1 

summaryData['stations'] = stations
summaryData['station_count'] = stationCount

# with open(outputPath, 'w') as jsonfile:
#     jsonfile.write(json.dumps(summaryData, sort_keys=True, indent=2))
# jsonfile.close

# Integrate client side processing to store state changes in last days
This is python code translated from JavaScript code from https://github.com/GeoGateway/geogateway-portal/blob/master/html/js/gps-tools.js

In [14]:
import time

In [15]:
# global params
gpsStationState=["green","red","yellow","lightblue","blue"]

In [16]:
def convert_string_to_date(string):
#     return datetime.strptime(string, '%Y-%m-%d').now().date()
    return datetime.strptime(string, '%Y-%m-%d')

In [17]:
def checkDateForData(selectedDate,noDataDates):
    dataOnDate=True;
    #   Selected date is after the last no-data date.
    if selectedDate > convert_string_to_date(noDataDates[len(noDataDates) -1]['to']):
        dataOnDate=True;

    # Otherwise, check each no-data interval to see if the date falls within.
    else:
        for elem in noDataDates:
            startDate = convert_string_to_date(elem['from'])
            endDate = convert_string_to_date(elem['to'])

            if (startDate <= selectedDate) & (endDate >= selectedDate):
                dataOnDate=False
                break

    return dataOnDate

In [18]:
# Would be nice to throw an exception here 
def getPrecedingStateChange(selectedDate,statusChanges):
    
    stateLastDate = selectedDate
    latestPossibleDate=convert_string_to_date(statusChanges[len(statusChanges)-1]['date'])
    earliestPossibleDate=convert_string_to_date(statusChanges[0]['date'])

#     This should actually throw an erorr since there is no earlier state change.
    if(selectedDate <= earliestPossibleDate):
#         stateLastDate=earliestPossibleDate;
        stateLastDate=selectedDate
    
    elif (selectedDate >= latestPossibleDate):
        stateLastDate=latestPossibleDate
        
    else:
        for i, e in reversed(list(enumerate(statusChanges))):
            stateChangeDate1 = convert_string_to_date(statusChanges[i-1]['date'])
            stateChangeDate2 = convert_string_to_date(statusChanges[i]['date'])    
#             The last state change date to find is the one
#             on or before the curren date.
            if (selectedDate >= stateChangeDate1) & (selectedDate < stateChangeDate2):
                stateLastDate=stateChangeDate1
#                 Dates are in order, so we can stop
                break;
        
#         for(var i=statusChanges.length-1; i>0; i--) {
#             var stateChangeDate1=new Date(statusChanges[i-1].date);
#             var stateChangeDate2=new Date(statusChanges[i].date);
#             //The last state change date to find is the one
#             //on or before the curren date.
#             if(selectedDate >= stateChangeDate1 
#                && selectedDate < stateChangeDate2) {
#                 stateLastDate=stateChangeDate1;
#                 //Dates are in order, so we can stop
#                 break;
#             }
#         }
    return stateLastDate

In [19]:
def getStationState(date,gpsStation):
    #     theState=gpsStationState[0]  #This is the default.
    theState=0
    today=date
    lastMonth=today - timedelta(30)
    dayBefore=today - timedelta(1)

    statusChanges=gpsStation['status_changes']
    noDataDates=gpsStation['time_nodata']

    earliestDataDate = convert_string_to_date(gpsStation['start_date'])
    dataOnDate = checkDateForData(date,noDataDates)

    #     Hopefully this big if-else construction correctly captures all the case.
    #     It can be simplified later.

    #     Provided date is before any data available for that station, so state is light blue
    if (today < earliestDataDate):
    #         theState=gpsStationState[3]  #light blue
        theState=3
    #     print ("c1")


    #     Date falls within data range, there are no status changes, and data is available on date
    elif (today >= earliestDataDate) & (len(statusChanges)==0) & (dataOnDate==True):
    #         theState=gpsStationState[0]  # green
        theState=0
    #     print ("c2")

    #     Date falls within data range, there are no state changes, and no data on selected date.
    elif (today > earliestDataDate) & (len(statusChanges)==0) &(dataOnDate==False):
    #         theState=gpsStationState[3]  # light blue
        theState=3
    #     print ("c3")


    #     We have data on the date, but it proceeds the earliest state change
    elif (today < convert_string_to_date(statusChanges[0]['date'])):
    #         theState=gpsStationState[0]  # green
        theState=0
    #     print ("c4")

    #     See if the date falls within 1 day or 1 month of a state change.
    elif (len(statusChanges) > 0):
        #     print ("c5")
        #    Get nearest preceding state change date
        stateLastDate=getPrecedingStateChange(date,statusChanges)

    #         See if state change was yesterday.
        if(stateLastDate > dayBefore):
            #             theState=gpsStationState[1]  # red
            theState=1

    #         See if the station has changed state in between the
    #         selected date and 30 days prior to the selected date.
        elif (stateLastDate >= lastMonth) & (stateLastDate <= today):
            #       See if we have no data within 24 hours of the selected date.            
            if (dataOnDate == False):
                #       Data is missing within last 24 hours and state has changed within last 30 days.
                #                 theState=gpsStationState[4]  # blue
                theState=4
            else:
                #       We have data on the date and state has changed within a 30 day window.
                #                 theState=gpsStationState[2] # yellow
                theState=2
        #         No data is available on selected date for this station, and station is
        #         not within either the 1 day or 30 day window.
        elif(dataOnDate==False):
            #             theState=gpsStationState[3]  # light blue
            theState=3

    return theState

# SAVE TO CASSANDRA

In [20]:
from cassandra.cluster import Cluster
from cassandra.policies import TokenAwarePolicy
from cassandra.policies import DCAwareRoundRobinPolicy
from cassandra.policies import RetryPolicy
# import cql

In [21]:
# from cassandra.cluster import Cluster
cluster = Cluster(contact_points=['127.0.0.1'], \
                  port=9042, \
                  load_balancing_policy= TokenAwarePolicy(DCAwareRoundRobinPolicy(local_dc='datacenter1')), \
                  default_retry_policy = RetryPolicy()
                 )

In [22]:
session = cluster.connect('system')

In [23]:
rows = cluster.metadata.keyspaces.keys()
rows

[u'system_traces',
 u'system_schema',
 u'system_auth',
 u'system',
 u'system_distributed']

In [24]:
dataSet = 'UNR_SPLICE'
KEYSPACE='GPS_'+dataSet
KEYSPACE=KEYSPACE.lower()
KEYSPACE

'gps_unr_splice'

In [25]:
session.default_timeout

10.0

## Drop existing database and create new database

In [26]:
keyspace_drop_stmt = session.prepare(
    "DROP KEYSPACE " + KEYSPACE
)

if KEYSPACE in rows:
    print ("dropping existing keyspace...")
    session.execute(keyspace_drop_stmt)

In [27]:
print("creating keyspace...")
session.execute("""
    CREATE KEYSPACE %s
    WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
    """ % KEYSPACE)

creating keyspace...


In [28]:
# print("setting keyspace...")
# connection.setup(['127.0.0.1'], KEYSPACE)
# connection.setup(['127.0.0.1'], KEYSPACE, protocol_version=3)

In [29]:
print("setting keyspace...")
session = cluster.connect(KEYSPACE)
# session.set_keyspace(KEYSPACE)

setting keyspace...


In [30]:
# session.execute("""
#     CREATE TYPE point (
#         lon double,
#         lat double);
#     """)

In [31]:
print("creating table...")
session.execute("""
    CREATE TABLE time_series_stations (
        date timestamp,
        station_id text,
        status int,
        PRIMARY KEY (date, station_id)
    )
""")

#     WITH CLUSTERING ORDER BY (date DESC);

creating table...


In [32]:
session.execute("""
    CREATE TABLE meta_stations (
        station_id text PRIMARY KEY,
        pro_dir text,
        "AFile" text,
        "BFile" text,
        "InputFile" text,
        "RawInputFile" text,
        "SwfInputFile" text,
        "DygraphsInputFile" text,
        "LFile" text,
        "XPngFile" text,
        "YPngFile" text,
        "ZPngFile" text,
        "XTinyPngFile" text,
        "YTinyPngFile" text,
        "ZTinyPngFile" text,
        "PiFile" text,
        "QFile" text,
        "MaxValFile" text,
        "MinValFile" text,
        "RangeFile" text,
        "ModelFiles" text,
        "RefFile" text,
        start_date timestamp,
        lat double,
        lon double,
        height double
    );
""")

print "creating index..."
session.execute("""
    CREATE INDEX on %s .meta_stations (lat);
""" % KEYSPACE)

session.execute("""
    CREATE INDEX on %s .meta_stations (lon);
""" % KEYSPACE)

#         loc frozen<point>

creating index...


In [33]:
session.execute("""
    CREATE TABLE meta_network (
        network_name text PRIMARY KEY,
        update_time timestamp,
        data_source text,
        begin_date timestamp,
        end_date timestamp,
        center_longitude double,
        center_latitude double,
        server_url text,
        "stateChangeNumTxtFile" text,
        "stateChangeNumJsInput" text,
        "allStationInputName" text,
        "Filters" text,
        video_url text,
        station_count int
    );
""")

<b>Alternative implementation using the Python wrapper for Cassandra DB</b>

In [34]:
# session.execute("""
#     DROP TABLE gps_unr_splice.time_series_stations;
# """)

In [35]:
# from cassandra.cqlengine import connection
# # # Connect to the demo keyspace on our cluster running at 127.0.0.1
# connection.setup(['127.0.0.1'], 'system', retry_connect=True)
# session = connection.session

In [36]:
# session.cluster.metadata.keyspaces.keys()

In [37]:
# if KEYSPACE in connection.cluster.metadata.keyspaces.keys():
#     print 'Dropping existing keyspace...'
#     session.execute("DROP KEYSPACE " + KEYSPACE)

    
# print 'Creating keyspace...'
# session.execute("""
#     CREATE KEYSPACE %s
#     WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
#     """ % KEYSPACE)

# print("setting keyspace...")
# session.set_keyspace(KEYSPACE)

In [38]:
# from cassandra.cqlengine import columns
# from cassandra.cqlengine.columns import *
# from cassandra.cqlengine.models import Model 
# from cassandra.cqlengine.usertype import UserType

# from cassandra.cqlengine import connection
# # from cqlengine import columns
# # from cqlengine.models import Model
# # from cqlengine import connection

# from datetime import datetime

In [39]:
# # /https://datastax.github.io/python-driver/cqlengine/models.html#user-types
# class Loc(UserType):
#     lon = Double()
#     lat = Double()

# class Time_series_stations(Model):
#     station_id = columns.Text(primary_key=True)
#     date = columns.DateTime(primary_key=True, clustering_order="DESC")
#     status = columns.Integer()    
    
# class Meta_stations(Model):
#     stations_id  = columns.Text(primary_key=True)
#     pro_dir = columns.Text()
#     AFile = columns.Text()
#     BFile = columns.Text()
#     InputFile = columns.Text()
#     RawInputFile = columns.Text()
#     SwfInputFile = columns.Text()
#     DygraphsInputFile = columns.Text()
#     LFile = columns.Text()
#     XPngFile = columns.Text()
#     YPngFile = columns.Text()
#     ZPngFile = columns.Text()
#     XTinyPngFile = columns.Text()
#     YTinyPngFile = columns.Text()
#     ZTinyPngFile = columns.Text()
#     PiFile = columns.Text()
#     QFile = columns.Text()
#     MaxValFile = columns.Text()
#     MinValFile = columns.Text()
#     RangeFile = columns.Text()
#     ModelFiles = columns.Text()
#     RefFile = columns.Text()
#     start_date = columns.DateTime()
#     loc = UserDefinedType(Loc)
#     height = columns.Text()
    
# class Meta_network(Model):
#     network_name = columns.Text(primary_key=True)
#     update_time= columns.DateTime()
#     data_source= columns.Text()
#     begin_date= columns.DateTime()
#     end_date = columns.DateTime()
#     center_longitude = columns.Double()
#     center_latitude= columns.Double()
#     server_url= columns.Text()
#     stateChangeNumTxtFile= columns.Text()
#     stateChangeNumJsInput= columns.Text()
#     allStationInputName= columns.Text()
#     Filters= columns.Text()
#     video_url= columns.Text()
#     station_count= columns.Integer()
    
# # users.create(name="Joe", addr=address(street="Easy St.", zipcode=99999))
# # user = users.objects(name="Joe")[0]
# # print user.name, user.addr
# # # Joe address(street=u'Easy St.', zipcode=99999)

In [40]:
# from cassandra.cqlengine.management import sync_table
# sync_table(Time_series_stations)
# sync_table(Meta_stations)
# sync_table(Meta_network)
# # sync_table(Time_series_stations)

# # sync_table(Time_series_stations, keyspaces=(KEYSPACE, ))

In [41]:
from datetime import datetime

In [42]:
# meta_stations_columns = "AFile,BFile,DygraphsInputFile,InputFile,LFile,MaxValFile, MinValFile, ModelFiles, PiFile,QFile,RangeFile,RawInputFile,RefFile,SwfInputFile,XPngFile,XTinyPngFile,YPngFile,YTinyPngFile,ZPngFile,ZTinyPngFile,height,station_id,lat,lon,pro_dir,start_date"

In [1]:
def get_station_element_as_list(station):
    dictionary = station.copy()
    dictionary.pop('status_changes', None)
    dictionary.pop('time_nodata', None)
    dictionary.pop('change_count', None)
    dictionary.pop('nodata_count', None)
    dictlist = []
    for key, value in sorted(dictionary.iteritems()):
#         print key, " : " ,value
        if (key == 'height') | (key == 'lat') | (key == 'long'):
            dictlist.append(float(value))
        elif key == 'start_date':
            dictlist.append(datetime.strptime(value, "%Y-%m-%d"))
        else:
            dictlist.append(value)

    return dictlist


In [44]:
start_date=datetime.strptime(beginDate, '%Y-%m-%d')
end_date=datetime.strptime(endDate, '%Y-%m-%d')

In [45]:
# http://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
from datetime import timedelta, date
# http://www.datastax.com/dev/blog/4-simple-rules-when-using-the-datastax-drivers-for-cassandra
from cassandra.query import BatchStatement

timeseries_insert_stmt = session.prepare("""
        INSERT INTO Time_series_stations
        (date, station_id, status) 
        VALUES (?, ?, ?)
""")


meta_stations_insert_stmt = session.prepare("""
    INSERT INTO meta_stations
    ("AFile","BFile","DygraphsInputFile",
    "InputFile","LFile","MaxValFile", 
    "MinValFile", "ModelFiles", 
    "PiFile","QFile","RangeFile",
    "RawInputFile","RefFile",
    "SwfInputFile","XPngFile",
    "XTinyPngFile","YPngFile",
    "YTinyPngFile","ZPngFile",
    "ZTinyPngFile",height,
    station_id,lat,lon,
    pro_dir,start_date)
    VALUES (?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, ?)
""")


meta_network_insert_stmt = session.prepare("""
    INSERT INTO meta_network
    (network_name, "Filters", "allStationInputName",
    begin_date, center_latitude, center_longitude, 
    data_source, end_date, server_url, 
    "stateChangeNumJsInput","stateChangeNumTxtFile", 
    station_count, update_time, video_url)
    VALUES (?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, 
            ?, ?, ?, ?)   
""")

In [46]:
meta_network_list = [dataSet, \
                     summaryData['Filters'], \
                     summaryData['allStationInputName'], \
                     datetime.strptime(summaryData['begin_date'], "%Y-%m-%d"), \
                     float(summaryData['center_latitude']), \
                     float(summaryData['center_longitude']), \
                     summaryData['data_source'], \
                     datetime.strptime(summaryData['end_date'], "%Y-%m-%d"), \
                     summaryData['server_url'], \
                     summaryData['stateChangeNumJsInput'], \
                     summaryData['stateChangeNumTxtFile'], \
                     int(summaryData['station_count']), \
                     datetime.strptime(summaryData['update_time'], "%Y-%m-%dT%H:%M:%S"), \
                     summaryData['video_url']
                    ]
# meta_network_list

In [47]:
session.execute(meta_network_insert_stmt, meta_network_list)

Although, it is suggested to execute the queries in batch, when deployed in production there are too many queries to process and Cassandra DB fails to execute all the insert queries

In [48]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
for single_date in daterange(start_date, end_date):
#     batch = BatchStatement()
    
    for station in stations:
        session.execute(timeseries_insert_stmt, \
          [single_date, station['id'],getStationState(single_date, station)])        
#         batch.add(timeseries_insert_stmt, \
#           [single_date, station['id'],getStationState(single_date, station)])
#         https://datastax.github.io/python-driver/user_defined_types.html
#         session.execute(meta_stations_insert_stmt, \
#           get_station_element_as_list(station))

    # execute the batch
#     session.execute(batch)
#         Time_series_stations.create(station_id=station['id'], \
#                                 date=single_date, \
#                                status=getStationState(single_date, station))

In [49]:
batch = BatchStatement()

for station in stations:
#     print station['id']
    
    batch.add(meta_stations_insert_stmt, \
          get_station_element_as_list(station))
    
session.execute(batch)

## Query database

### TIME SERIES QUERY

In [50]:
# https://datastax.github.io/python-driver/faq.html
# https://datastax.github.io/python-driver/api/cassandra/query.html
from cassandra.query import named_tuple_factory
session.row_factory = named_tuple_factory

lon_min=-230
lon_max=-90
lat_min=-13
lat_max=48

year= 2013
month=12
day=31

# http://www.datastax.com/dev/blog/allow-filtering-explained-2
# http://stackoverflow.com/questions/28250534/cqlengine-queries-how-to-return-dictionary
meta_stations_select_stmt = session.prepare("""
    select station_id, lat, lon from meta_stations 
    where lon >= ? and lon <= ? and lat >= ? and lat <= ?
    allow FILTERING
""")

selected_stations = session.execute(meta_stations_select_stmt, \
                        [lon_min, lon_max, lat_min, lat_max])

In [51]:
selected_stations.current_rows

[Row(station_id=u'ESM1', lat=47.803480434, lon=-122.569173848),
 Row(station_id=u'00NA', lat=-12.466640097, lon=-229.156012745),
 Row(station_id=u'1NSU', lat=31.75080049, lon=-93.097603682),
 Row(station_id=u'01NA', lat=-12.478223997, lon=-229.017953041),
 Row(station_id=u'1LSU', lat=30.40742467, lon=-91.180261768)]

In [52]:
futures = []
timeseries_select_stmt = session.prepare("""
    SELECT status  
    FROM time_series_stations
    WHERE date = ? AND station_id = ?
""")

list_station = list()
for e in list(selected_stations):
    res ={ 'station_id' : str(e.station_id), \
         'lat' : str(e.lat), \
         'lon' : str(e.lon) \
         }
    list_station.append(res)
    futures.append(session.execute_async(timeseries_select_stmt, \
                              [datetime(year,month,day), str(e.station_id)]))


In [53]:
# wait for them to complete and use the results

for idx, future in enumerate(futures):
    record = future.result()
    
#     print list(record)
    t = list_station[idx]
    t['status']= str(list(record)[0][0])
    
list_station

[{'lat': '47.803480434',
  'lon': '-122.569173848',
  'station_id': 'ESM1',
  'status': '3'},
 {'lat': '-12.466640097',
  'lon': '-229.156012745',
  'station_id': '00NA',
  'status': '3'},
 {'lat': '31.75080049',
  'lon': '-93.097603682',
  'station_id': '1NSU',
  'status': '0'},
 {'lat': '-12.478223997',
  'lon': '-229.017953041',
  'station_id': '01NA',
  'status': '3'},
 {'lat': '30.40742467',
  'lon': '-91.180261768',
  'station_id': '1LSU',
  'status': '0'}]

### NETWORK QUERY

In [54]:
from cassandra.query import dict_factory
session.row_factory = dict_factory

In [55]:
meta_network_select_stmt = session.prepare("""
    select * from meta_network 
""")

In [56]:
nw = session.execute(meta_network_select_stmt)

In [57]:
nw[0]

{u'Filters': u'Fill_Missing',
 u'allStationInputName': u'all_stations.all.input',
 u'begin_date': datetime.datetime(1994, 1, 1, 0, 0),
 u'center_latitude': 36.7477778,
 u'center_longitude': -119.7713889,
 u'data_source': u'UNR_SPLICE',
 u'end_date': datetime.datetime(2016, 10, 19, 0, 0),
 u'network_name': u'UNR_SPLICE',
 u'server_url': u'http://gf9.ucs.indiana.edu/daily_rdahmmexec/daily/UNR_SPLICE',
 u'stateChangeNumJsInput': u'stateChangeNums.txt.jsi',
 u'stateChangeNumTxtFile': u'stateChangeNums.txt',
 u'station_count': 5,
 u'update_time': datetime.datetime(2016, 11, 20, 21, 4, 40),
 u'video_url': ''}

### STATION META DATA

In [58]:
station_id_to_find='00NA'

meta_stations_select_stmt = session.prepare("""
    select * from meta_stations 
    where station_id = ?
""")

selected_station = session.execute(meta_stations_select_stmt, \
                        [station_id_to_find])

In [59]:
output = selected_station[0]
output

{u'AFile': u'daily_project_00NA.A',
 u'BFile': u'daily_project_00NA.B',
 u'DygraphsInputFile': u'daily_project_00NA_2016-10-19.dygraphs.js',
 u'InputFile': u'daily_project_00NA_2016-10-19.all.input',
 u'LFile': u'daily_project_00NA.L',
 u'MaxValFile': u'daily_project_00NA.maxval',
 u'MinValFile': u'daily_project_00NA.minval',
 u'ModelFiles': u'daily_project_00NA.zip',
 u'PiFile': u'daily_project_00NA.pi',
 u'QFile': u'daily_project_00NA_2016-10-19.all.Q',
 u'RangeFile': u'daily_project_00NA.range',
 u'RawInputFile': u'daily_project_00NA_2016-10-19.all.raw',
 u'RefFile': u'daily_project_00NA.input.ref',
 u'SwfInputFile': u'daily_project_00NA_2016-10-19.plotswf.input',
 u'XPngFile': u'daily_project_00NA_2016-10-19.all.input.X.png',
 u'XTinyPngFile': u'daily_project_00NA_2016-10-19.all.input.X_tiny.png',
 u'YPngFile': u'daily_project_00NA_2016-10-19.all.input.Y.png',
 u'YTinyPngFile': u'daily_project_00NA_2016-10-19.all.input.Y_tiny.png',
 u'ZPngFile': u'daily_project_00NA_2016-10-19.all.

In [60]:
# tp= session.prepare("""
#     SELECT * FROM time_series_stations
#     WHERE date = ?
# """)

# tf= session.execute(tp, \
#                    [datetime(2016, 5, 12)])

# list(tf)

# Close conenction to Cassandra db

In [ ]:
session.shutdown()

In [ ]:
cluster.shutdown()